In [ ]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

In [ ]:
from langchain_openai import OpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-instruct")
response=llm("explain large language models in one sentence")
print(response)

In [4]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI

In [ ]:
chat=ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0.3)
messages=[
    SystemMessage(content="You are an expert data scientist."),
    HumanMessage(content="Write a Python code to calculate the sum of two numbers.")
]
response=chat(messages)
print(response.content, end="\n\n")

In [6]:
from langchain import PromptTemplate
template = """
You are a data scientis working at a company. You are asked to write a Python code to calculate the sum of {content} numbers. Write the code below.
"""
prompt=PromptTemplate(
    input_variable="content",
    template=template
)

In [ ]:
print(prompt)

In [ ]:
response=llm(prompt.format(content="three"))
print(response)

In [ ]:
from langchain.chains import LLMChain
chain=LLMChain(llm=llm,prompt=prompt)
print(chain.run("two"))

In [11]:
second_prompt=PromptTemplate(
    input_variables=["content2"],
    template="Divide the sum {content2} by 3."
)
chain_two=LLMChain(llm=llm,prompt=second_prompt)

In [ ]:
from langchain.chains import SimpleSequentialChain
overall_chain=SimpleSequentialChain(chains=[chain, chain_two], verbose=True)
explanation=overall_chain.run("two")
print(explanation)

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=0
)

texts = text_splitter.create_documents(explanation)

In [ ]:
texts

In [ ]:
texts[0].page_content

In [ ]:
from langchain.embeddings import OpenAIEmbeddings

embeddings=OpenAIEmbeddings(model="text-embedding-ada-002")

In [ ]:
query_result=embeddings.embed_query(texts[0].page_content)
query_result

In [ ]:
import os
from pinecone import Pinecone

# Initialize Pinecone

my_api_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=my_api_key)  

index_name = "langchain-embeddings"
search = pc.from_documents(index_name, query_result)